In [2]:
from netCDF4 import Dataset
import numpy as np
import math
import cartopy
import cartopy.crs as ccrs
import matplotlib.pyplot as plt
import pandas as pd
from pandas.core.frame import DataFrame

In [11]:
def get_TROPOMI(file_path,longitude,latitude):
    ##The longitude and latitude variables are longitude and latitude of the pandora sites. 
    ##For BoulderCO it's longitude = -105.26 and latitude = 39.99, 
    ##for GreenbeltMD it's longitude = -76.8396, latitude = 38.9926,
    ##for MaunaLoaHI it's longitude = -155.6027, latitude = 19.4795,
    ##for NewHavenCT it's longitude = -72.9029, latitude = 41.3014
    ##I don't want to be so troublesome too, but I found that the four places can be contained in two different .nc files. :(
    f = Dataset(file_path, mode='r')
    lon = f.groups['PRODUCT'].variables['longitude'][0][:]
    lat = f.groups['PRODUCT'].variables['latitude'][0][:]
    time = f.groups['PRODUCT'].variables['time_utc'][0][:]
    units = f.groups['PRODUCT'].variables['nitrogendioxide_tropospheric_column'].units
    NO2_trop = f.groups['PRODUCT'].variables['nitrogendioxide_tropospheric_column'][0][:]
    NO2_col_df = DataFrame(NO2_trop)
    list_1 = []
    for i in range(0,len(NO2_trop)):
        for j in range(0,len(NO2_trop[0])):
            if lon[i][j] >= longitude-0.1802 and lon[i][j] <= longitude + 0.1802:
                if lat[i][j] >= latitude-0.1802 and lat[i][j] <= latitude+0.1802:
                    if math.isnan(NO2_col_df[j][i]) == False:
                        list_1.append((i,j))
    dist = []
    dist_min = (9999,9999)
    for i in list_1:
        dist.append((lon[i[0]][i[1]] + 105.26)**2 + (lat[i[0]][i[1]] - 39.99)**2)
    for i in range(len(dist)):
        if dist[i] == min(dist):
            dist_min = list_1[i]
    if dist_min == (9999,9999):
        print('all the qualified coincident data is NaN')
    lon_min = lon[dist_min[0]][dist_min[1]]
    lat_min = lat[dist_min[0]][dist_min[1]]
    NO2_col_min = NO2_col_df[dist_min[1]][dist_min[0]]
    time_min = time[dist_min[0]]
    return [lon_min, lat_min, NO2_col_min, units, time_min]

In [12]:
BoulderCO = get_TROPOMI("S5P_OFFL_L2__NO2____20191106T184032_20191106T202201_10703_01_010302_20191112T214725.nc",-105.26,39.99)

In [14]:
NewHavenCT = get_TROPOMI("S5P_OFFL_L2__NO2____20191106T165902_20191106T184032_10702_01_010302_20191112T191844.nc",-72.9029, 41.3014)

In [15]:
GreenbeltMD = get_TROPOMI("S5P_OFFL_L2__NO2____20191106T165902_20191106T184032_10702_01_010302_20191112T191844.nc",longitude = -76.8396, latitude = 38.9926)

In [17]:
MaunaLoaHI = []
##It seems that it's not contained in any of the four files.

In [18]:
BoulderCO[-1]

'2019-11-06T19:46:46.324000Z'

In [19]:
NewHavenCT

[-73.07848, 41.35316, 3.50898e-05, 'mol m-2', '2019-11-06T18:05:16.780000Z']

In [20]:
GreenbeltMD

[-77.01803, 39.07455, 4.6375877e-05, 'mol m-2', '2019-11-06T18:04:50.741000Z']